# Pasar datos en formato json a txt

Comenzamos importando las librerías y estableciendo las opciones de pandas que queremos.

In [380]:
import glob
import json
import pandas as pd
from datetime import datetime as dt
import re
import unidecode as ud
from IPython.display import Image

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None

El json obtenido con twarc2 tiene múltiples items, uno por cada 100 tweets. En este caso tenemos dos, los cogemos y los metemos en una lista.

In [381]:
tweets_data = []

tweets_file = open(r'felizmiercoles.json1', encoding = 'utf-8')

for line in tweets_file:
    tweet = json.loads(line)
    tweets_data.append(tweet)

tweets_file.close()

Vamos a comprobar que nuestro json tiene 2 items al tener 200 tweets.

In [382]:
len(tweets_data)

2

### Ejemplo de estructura de json

En este ejemplo solo he cogido un tweet del hashtag #estoesunaprueba123 para facilitar la visualización. Si tuvieramos más de 100 tweets el diccionario que marca el primer corchete contendría una lista de items.

In [383]:
Image(url="Imagenes/json_tree.png", width=600,height=600)

### Extracción de los datos

Loopeamos en la lista de items del json para coger cada pieza de información de todos los tweets e introducirla en una variable. Por último el loop va a unir toda la información en `list_tweets_flattered`, que es una lista vacía que vamos a llenar con otras listas, una por tweet.

In [384]:
list_tweets_flattered = []

for item in tweets_data:
    list_tweets = item['data']
    for tweet in list_tweets:
        if 'referenced_tweets' in tweet:
            type = tweet['referenced_tweets'][0]['type']
            if type == 'retweeted':
                user_retweeted = tweet['entities']['mentions'][0]['username']
        else:
            type = 'tweet'
            user_retweeted = None
        id_user = tweet['author_id']
        date = tweet['created_at']
        source = tweet['source']
        reply_settings = tweet['reply_settings']
        id_tweet = tweet['id']
        retweet_count = tweet['public_metrics']['retweet_count']
        reply_count = tweet['public_metrics']['reply_count']
        like_count = tweet['public_metrics']['like_count']
        quote_count = tweet['public_metrics']['quote_count']
        lang = tweet['lang']

        list_tweets_flattered.append([type,user_retweeted,id_user,date,source,reply_settings,id_tweet,retweet_count,
                                        reply_count,like_count,quote_count,lang])

Esa lista de listas la convertimos en dataframe.

In [385]:
tweets_df = pd.DataFrame(list_tweets_flattered,columns=['relation','user retweeted','id user','date','app','reply settings',
'id tweet','retweet count','reply count','like count','quote count','lang'])
tweets_df.head(2)

,relation,user retweeted,id user,date,app,reply settings,id tweet,retweet count,reply count,like count,quote count,lang
0,retweeted,Elangel_ven0,1280509194534346752,2022-10-19T15:15:26.000Z,Twitter Web App,everyone,1582752287495458817,43,0,0,0,es
1,retweeted,sergiofimbres,795089089,2022-10-19T15:15:25.000Z,Twitter for Android,everyone,1582752285595029505,7,0,0,0,es


Hacemos lo mismo con la información de los usuarios.

In [386]:
list_users_flattered = []

for item in tweets_data:
    list_users = item['includes']['users']
    for user in list_users:
        if 'location' in user:
            location = user['location']
        else:
            location = None
        username = user['username']
        verified = user['verified']
        name = user['name']
        id_user = user['id']
        profile_image_url = user['profile_image_url']
        followers_count = user['public_metrics']['followers_count']
        following_count = user['public_metrics']['following_count']
        tweet_count = user['public_metrics']['tweet_count']
        listed_count = user['public_metrics']['listed_count']
        bio = user['description']
        created_at = user['created_at']
        if 'url' in user:
            url_list = user['entities']['url']['urls']
            url_end = []
            for url_unique in url_list:
                expanded_url = url_unique['expanded_url']
                url_end.append(expanded_url)
        else:
            url_end = None

        list_users_flattered.append([location,username,verified,name,id_user,profile_image_url,
        followers_count,following_count,tweet_count,listed_count,bio,created_at,url_end])

In [387]:
users_df = pd.DataFrame(list_users_flattered,columns=['location','author','verified','name',
'id user','profile image url','followers','following','stauses','listed',
'description','created_at','urls'])

In [388]:
users_df.head(1)

,location,author,verified,name,id user,profile image url,followers,following,stauses,listed,description,created_at,urls
0,Apure,ShavelaHermosa,False,♥️⋆ 𝓒𝓱𝓪𝓿𝓮𝓵𝓪 𝓐𝓷𝓭𝓻𝓮𝓲𝓷𝓪 ⋆ ♥️,1280509194534346752,https://pbs.twimg.com/profile_images/1539401404896378880/X5TJ6zo2_normal.jpg,4402,3183,73680,3,"Llanera, me gusta viajar y conocer mi país 🇻🇪\n♫ ..En el fogón de tus brazos cocíname a fuego \nlento pàque mi alma se sancoche\n con el calor de tu cuerpo.♫",2020-07-07T14:29:32.000Z,None


Hacemos lo mismo con la fecha en la que hemos cogido los tweets.

In [389]:
retrieved_at = tweets_data[0]['__twarc']['retrieved_at']

Añadimos la información de los usuarios y la fecha de obtención al dataframe de tweets.

In [390]:
merged = tweets_df.merge(users_df,how='left',on='id user')
merged['retrieved at'] = retrieved_at

In [391]:
merged.head(1)

,relation,user retweeted,id user,date,app,reply settings,id tweet,retweet count,reply count,like count,quote count,lang,location,author,verified,name,profile image url,followers,following,stauses,listed,description,created_at,urls,retrieved at
0,retweeted,Elangel_ven0,1280509194534346752,2022-10-19T15:15:26.000Z,Twitter Web App,everyone,1582752287495458817,43,0,0,0,es,Apure,ShavelaHermosa,False,♥️⋆ 𝓒𝓱𝓪𝓿𝓮𝓵𝓪 𝓐𝓷𝓭𝓻𝓮𝓲𝓷𝓪 ⋆ ♥️,https://pbs.twimg.com/profile_images/1539401404896378880/X5TJ6zo2_normal.jpg,4402,3183,73680,3,"Llanera, me gusta viajar y conocer mi país 🇻🇪\n♫ ..En el fogón de tus brazos cocíname a fuego \nlento pàque mi alma se sancoche\n con el calor de tu cuerpo.♫",2020-07-07T14:29:32.000Z,None,2022-10-19T15:15:40+00:00


Limpiamos las columnas de fechas.

In [392]:
merged['date'] = merged['date'].apply(lambda x:str(x))
merged['date'] = merged['date'].str.replace('T',' ')
merged['date'] = merged['date'].str.replace('Z','')
merged['date'] = merged['date'].str.replace('000','')
merged['date'] = merged['date'].str.replace('.','')

C:\Users\Piratena\AppData\Local\Temp\ipykernel_13760\2984507682.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  merged['date'] = merged['date'].str.replace('.','')


In [393]:
merged['created_at'] = merged['created_at'].apply(lambda x:str(x))
merged['created_at'] = merged['created_at'].str.replace('T',' ')
merged['created_at'] = merged['created_at'].str.replace('Z','')
merged['created_at'] = merged['created_at'].str.replace('000','')
merged['created_at'] = merged['created_at'].str.replace('.','')

C:\Users\Piratena\AppData\Local\Temp\ipykernel_13760\1595034468.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  merged['created_at'] = merged['created_at'].str.replace('.','')


In [394]:
merged['retrieved at'] = merged['retrieved at'].str.replace('T',' ').apply(lambda x:str(x)).apply(lambda x:x[:-6])

Transformamos las 3 columnas de fechas de nuevo a objeto datetime (fecha y tiempo).

In [395]:
merged['date'] = merged['date'].apply(lambda x:dt.strptime(x, '%Y-%m-%d %H:%M:%S'))
merged['created_at'] = merged['created_at'].apply(lambda x:dt.strptime(x, '%Y-%m-%d %H:%M:%S'))
merged['retrieved at'] = merged['retrieved at'].apply(lambda x:dt.strptime(x, '%Y-%m-%d %H:%M:%S'))

Añadimos la columna del texto.

In [396]:
text_csv = pd.read_csv('felizmiercoles_csv.txt')
text_csv = text_csv[['id','text']]
text_csv = text_csv.rename(columns={'id':'id tweet'})
text_csv

,id tweet,text
0,1582752287495458817,"YO♥️CUMANA\n🚩 Tras las lluvias acaecidas en #Cumana el alcalde 👉 .@lossifontes realizo el acercamiento a la comunidad de El ""Tacal sector 2"" y atendió las afectaciones, de la mano con el pueblo✌️\n#19Oct\n#PuebloUnidoYActivo\n#FelizMiercoles\n#Sucre\n▶️ .@NicolasMaduro https://t.co/d7r1MQhpdP"
1,1582752285595029505,📰 Segob vs ego (@osvaldomonos) \n#FelizMiercoles https://t.co/mzLhRFzLew
2,1582752266939138049,#FelizMiércoles hoy sera un excelente día!! https://t.co/uUO20FXDIN
3,1582752259091595267,Que sea hoy un día lleno de bendiciones!\n#FelizMiercoles https://t.co/IRB3FDUcBM
4,1582752255454851072,"#BuenosDíasATodos ☕️🫓🗞️🌳🇻🇪🏔️💼\n\n#FelizMiercoles #19Oct\n#1x10DelBuenGobierno #PuebloUnidoYActivo\n\n""En un mundo de plástico y ruido, quiero ser de barro y de silencio"". Eduardo Galeano\n\nEstatua San Fernando- Estado #Apure https://t.co/qeBz5j3hMu"
...,...,...
195,1582750144381919232,@Sara_VA_ Que belleza! Sara... #FelizMiércoles Buenos dias https://t.co/uiyYLA3Oon
196,1582750138837372928,¡Buenos días!\n#FelizMiercoles https://t.co/t1dGHbnAXx
197,1582750128498421760,#FelizMiercoles https://t.co/aIOrnAvAZX
198,1582750125528875009,"así mismo, nubosidad fragmentada en áreas del Táchira, Barinas, Lara, Yaracuy, este de Miranda, Anzoátegui y Monagas; el resto del país se aprecia zonas despejadas. #Reporte de las 04:40 HLV #FelizMiércoles"


In [397]:
merged['description'].str.replace('\n',' ')

0      Llanera, me gusta viajar y conocer mi país 🇻🇪 ♫ ..En el fogón de tus brazos cocíname a fuego  lento pàque mi alma se sancoche  con el calor de tu cuerpo.♫
1                                                                                                                                                                
2                                                                                                                                                                
3                                                                                                                                                                
4                                                                                                                                  Chavez Vive ... La Lucha Sigue
                                                                                  ...                                                                            
211                         

In [398]:
text_csv['text'].str.replace('\n',' ')

0      YO♥️CUMANA\n🚩 Tras las lluvias acaecidas en #Cumana el alcalde 👉 .@lossifontes realizo el acercamiento a la comunidad de El "Tacal sector 2" y atendió las afectaciones, de la mano con el pueblo✌️\n#19Oct\n#PuebloUnidoYActivo\n#FelizMiercoles\n#Sucre\n▶️ .@NicolasMaduro https://t.co/d7r1MQhpdP
1                                                                                                                                                                                                                                   📰 Segob vs ego (@osvaldomonos) \n#FelizMiercoles https://t.co/mzLhRFzLew
2                                                                                                                                                                                                                                        #FelizMiércoles hoy sera un excelente día!! https://t.co/uUO20FXDIN
3                                                                                                

In [399]:
merged['id tweet'] = merged['id tweet'].apply(lambda x: int(x))
merged_text = merged.merge(text_csv,how='left',on='id tweet')

Ordenamos el dataframe de tweets más antiguos a más recientes.

In [400]:
merged_text = merged_text.sort_values(by='date',ascending=True)

Quitamos los saltos de línea de las columnas de textos largos.

In [401]:
merged_text['text'] = merged_text['text'].apply(lambda x:str(x))

In [402]:
merged_text['text'] = merged_text['text'].replace('\n',' ')
merged_text['description'] = merged_text['description'].str.replace('\n',' ')

Quitamos tildes.

In [403]:
merged_text['text'] = merged_text['text'].apply(lambda x:ud.unidecode(x))

In [404]:
prueba = '12\n#FelizMiercoles https://t.co/2k3ETNfzng'
prueba.replace('\n',' ')

'12 #FelizMiercoles https://t.co/2k3ETNfzng'

In [405]:
merged_text

,relation,user retweeted,id user,date,app,reply settings,id tweet,retweet count,reply count,like count,quote count,lang,location,author,verified,name,profile image url,followers,following,stauses,listed,description,created_at,urls,retrieved at,text
215,tweet,None,296592711,2022-10-19 15:06:43,Twitter Web App,everyone,1582750095019499520,0,0,0,0,und,CDMX,jramiroMX,False,CHILANGO,https://pbs.twimg.com/profile_images/1311646474984476672/LjlH6TKT_normal.jpg,188356,209256,525227,345,"Que nadie manipule tu criterio, ¿como puedes evitarlo?, solo adquiriendo el buen hábito de estar bien informado, no existe otra forma",2011-05-11 02:05:17,None,2022-10-19 15:15:40,12\n#FelizMiercoles https://t.co/2k3ETNfzng
214,retweeted,INAMEH,1582023931250282497,2022-10-19 15:06:50,Twitter for Android,everyone,1582750125528875009,127,0,0,0,es,None,carmen21463660,False,carmen,https://pbs.twimg.com/profile_images/1582024161647550466/FOKc72l5_normal.jpg,0,40,533,0,,2022-10-17 15:01:30,None,2022-10-19 15:15:40,"asi mismo, nubosidad fragmentada en areas del Tachira, Barinas, Lara, Yaracuy, este de Miranda, Anzoategui y Monagas; el resto del pais se aprecia zonas despejadas. #Reporte de las 04:40 HLV #FelizMiercoles"
213,retweeted,AndaluciaSinVOX,587308440,2022-10-19 15:06:51,Twitter for iPhone,everyone,1582750128498421760,225,0,0,0,qme,None,mariajosepascu3,False,maria jose pascual,https://pbs.twimg.com/profile_images/907287465354035200/wcAkBhFU_normal.jpg,825,918,105757,11,,2012-05-22 07:31:09,None,2022-10-19 15:15:40,#FelizMiercoles https://t.co/aIOrnAvAZX
212,retweeted,SaulGtrauma,1554130285800849410,2022-10-19 15:06:53,Twitter Web App,everyone,1582750138837372928,5,0,0,0,es,None,nathagrullon1,False,Natha Grullon,https://pbs.twimg.com/profile_images/1554130428172402689/prA8Ns7V_normal.jpg,27,48,989,0,Arquitecto Grullon. Da lo mejor de ti siempre.,2022-08-01 15:41:58,None,2022-10-19 15:15:40,!Buenos dias!\n#FelizMiercoles https://t.co/t1dGHbnAXx
211,replied_to,BlackMambaSuite,1022323066641305600,2022-10-19 15:06:55,Twitter for Android,everyone,1582750144381919232,0,0,0,0,es,"Guadalajara, Jalisco",Ricardo63207717,False,RICHIE ⚽ #soydepumas 💙👊// #halamadrid 🏈 Raiders,https://pbs.twimg.com/profile_images/1577535197666492416/9yg1pQqa_normal.jpg,2882,5003,57586,3,https://t.co/pzM5qE1BLS 54 🎂 💓🍀Que valga cada dia 🎲 🎩 #ADARLE 🥃 👊 🇲🇽 Hebreos 13: 5,2018-07-26 03:29:47,None,2022-10-19 15:15:40,@Sara_VA_ Que belleza! Sara... #FelizMiercoles Buenos dias https://t.co/uiyYLA3Oon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,retweeted,MObras_Publicas,1224625887993376769,2022-10-19 15:15:18,Twitter for Android,everyone,1582752255454851072,8,0,0,0,es,Venezuela,IsabelTeresaSu4,False,🎄💫 @IsabelTeresaSu4 💫,https://pbs.twimg.com/profile_images/1573818086469390336/p5v3CSpU_normal.jpg,1248,1227,65797,3,Chavez Vive ... La Lucha Sigue,2020-02-04 09:29:25,None,2022-10-19 15:15:40,"#BuenosDiasATodos \n\n#FelizMiercoles #19Oct\n#1x10DelBuenGobierno #PuebloUnidoYActivo\n\n""En un mundo de plastico y ruido, quiero ser de barro y de silencio"". Eduardo Galeano\n\nEstatua San Fernando- Estado #Apure https://t.co/qeBz5j3hMu"
3,retweeted,Motivos14185270,1323795044889817089,2022-10-19 15:15:19,Twitter for Android,everyone,1582752259091595267,369,0,0,0,es,México,Lauramonroy33,False,Laura Monroy,https://pbs.twimg.com/profile_images/1575641140245692416/ugyYN9TE_normal.jpg,63,27,252,0,,2020-11-04 01:12:05,None,2022-10-19 15:15:40,Que sea hoy un dia lleno de bendiciones!\n#FelizMiercoles https://t.co/IRB3FDUcBM
2,retweeted,RocioRo55,1580963194448842754,2022-10-19 15:15:21,Twitter for Android,everyone,1582752266939138049,1,0,0,0,es,None,RocioRo55,False,Rocio Rodriguez,https://pbs.twimg.com/profile_images/1581067682048245766/H4OEiuFD_normal.jpg,82,102,42,0,,2022-10-14 16:46:42,None,2022-10-19 15:15:40,#FelizMiercoles hoy sera un excelente dia!! https://t.co/uUO20FXDIN
1,retweeted,sergiofimbres,795089089,2022-10-19 15:15:25,Twitter f

Añadimos columnas.

In [406]:
merged_text['geolocation'] = 'None'
merged_text['first HT'] = merged_text['text'].apply(lambda x: re.findall(r"#(\w+)", x))

Ordenamos las columnas.

In [407]:
merged_text = merged_text[['id tweet','date','author','text','app','id user','followers','following',
'stauses','location','urls','geolocation','name','description','relation','user retweeted',
'first HT','lang','created_at']]

In [408]:
merged_text['first HT'] = merged_text['first HT'].apply(lambda x:x[0])

In [409]:
merged_text.head(2)

,id tweet,date,author,text,app,id user,followers,following,stauses,location,urls,geolocation,name,description,relation,user retweeted,first HT,lang,created_at
215,1582750095019499520,2022-10-19 15:06:43,jramiroMX,12\n#FelizMiercoles https://t.co/2k3ETNfzng,Twitter Web App,296592711,188356,209256,525227,CDMX,None,None,CHILANGO,"Que nadie manipule tu criterio, ¿como puedes evitarlo?, solo adquiriendo el buen hábito de estar bien informado, no existe otra forma",tweet,None,FelizMiercoles,und,2011-05-11 02:05:17
214,1582750125528875009,2022-10-19 15:06:50,carmen21463660,"asi mismo, nubosidad fragmentada en areas del Tachira, Barinas, Lara, Yaracuy, este de Miranda, Anzoategui y Monagas; el resto del pais se aprecia zonas despejadas. #Reporte de las 04:40 HLV #FelizMiercoles",Twitter for Android,1582023931250282497,0,40,533,None,None,None,carmen,,retweeted,INAMEH,Reporte,es,2022-10-17 15:01:30


In [410]:
merged_text['text'] = merged_text['text'].str.replace('\n',' ')

In [411]:
merged_text

,id tweet,date,author,text,app,id user,followers,following,stauses,location,urls,geolocation,name,description,relation,user retweeted,first HT,lang,created_at
215,1582750095019499520,2022-10-19 15:06:43,jramiroMX,12\n#FelizMiercoles https://t.co/2k3ETNfzng,Twitter Web App,296592711,188356,209256,525227,CDMX,None,None,CHILANGO,"Que nadie manipule tu criterio, ¿como puedes evitarlo?, solo adquiriendo el buen hábito de estar bien informado, no existe otra forma",tweet,None,FelizMiercoles,und,2011-05-11 02:05:17
214,1582750125528875009,2022-10-19 15:06:50,carmen21463660,"asi mismo, nubosidad fragmentada en areas del Tachira, Barinas, Lara, Yaracuy, este de Miranda, Anzoategui y Monagas; el resto del pais se aprecia zonas despejadas. #Reporte de las 04:40 HLV #FelizMiercoles",Twitter for Android,1582023931250282497,0,40,533,None,None,None,carmen,,retweeted,INAMEH,Reporte,es,2022-10-17 15:01:30
213,1582750128498421760,2022-10-19 15:06:51,mariajosepascu3,#FelizMiercoles https://t.co/aIOrnAvAZX,Twitter for iPhone,587308440,825,918,105757,None,None,None,maria jose pascual,,retweeted,AndaluciaSinVOX,FelizMiercoles,qme,2012-05-22 07:31:09
212,1582750138837372928,2022-10-19 15:06:53,nathagrullon1,!Buenos dias!\n#FelizMiercoles https://t.co/t1dGHbnAXx,Twitter Web App,1554130285800849410,27,48,989,None,None,None,Natha Grullon,Arquitecto Grullon. Da lo mejor de ti siempre.,retweeted,SaulGtrauma,FelizMiercoles,es,2022-08-01 15:41:58
211,1582750144381919232,2022-10-19 15:06:55,Ricardo63207717,@Sara_VA_ Que belleza! Sara... #FelizMiercoles Buenos dias https://t.co/uiyYLA3Oon,Twitter for Android,1022323066641305600,2882,5003,57586,"Guadalajara, Jalisco",None,None,RICHIE ⚽ #soydepumas 💙👊// #halamadrid 🏈 Raiders,https://t.co/pzM5qE1BLS 54 🎂 💓🍀Que valga cada dia 🎲 🎩 #ADARLE 🥃 👊 🇲🇽 Hebreos 13: 5,replied_to,BlackMambaSuite,FelizMiercoles,es,2018-07-26 03:29:47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,1582752255454851072,2022-10-19 15:15:18,IsabelTeresaSu4,"#BuenosDiasATodos \n\n#FelizMiercoles #19Oct\n#1x10DelBuenGobierno #PuebloUnidoYActivo\n\n""En un mundo de plastico y ruido, quiero ser de barro y de silencio"". Eduardo Galeano\n\nEstatua San Fernando- Estado #Apure https://t.co/qeBz5j3hMu",Twitter for Android,1224625887993376769,1248,1227,65797,Venezuela,None,None,🎄💫 @IsabelTeresaSu4 💫,Chavez Vive ... La Lucha Sigue,retweeted,MObras_Publicas,BuenosDiasATodos,es,2020-02-04 09:29:25
3,1582752259091595267,2022-10-19 15:15:19,Lauramonroy33,Que sea hoy un dia lleno de bendiciones!\n#FelizMiercoles https://t.co/IRB3FDUcBM,Twitter for Android,1323795044889817089,63,27,252,México,None,None,Laura Monroy,,retweeted,Motivos14185270,FelizMiercoles,es,2020-11-04 01:12:05
2,1582752266939138049,2022-10-19 15:15:21,RocioRo55,#FelizMiercoles hoy sera un excelente dia!! https://t.co/uUO20FXDIN,Twitter for Android,1580963194448842754,82,102,42,None,None,None,Rocio Rodriguez,,retweeted,RocioRo55,FelizMiercoles,es,2022-10-14 16:46:42
1,1582752285595029505,2022-10-19 15:15:25,charly692009,Segob vs ego (@osvaldomonos) \n#FelizMiercoles https://t.co/mzLhRFzLew,Twitter for Android,795089089,94,400,47292,"Monterrey, Nuevo León",None,None,Carlos Guadarrama,,retweeted,sergiofimbres,FelizMiercoles,es,2012-09-01 01:48:06


Limpiamos los duplicados que se hayan podido generar.

In [412]:
merged_text = merged_text.drop_duplicates(subset='id tweet',keep='first')

Lo guardamos como .txt con separación de tabulador, que es el que usa t-hoarder.

In [413]:
merged_text.to_csv('felizmiercoles.txt',index=False,sep='\t')